<a href="https://colab.research.google.com/github/mukunthans/DLAssignment2/blob/main/DLAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = '/content/drive/MyDrive/inaturalist_12K'


In [ ]:
pip install pytorch-lightning


In [ ]:
import os
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torch.nn.functional as F
from torch import nn
import torchvision.models as models


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'val'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from pytorch_lightning import LightningModule

# 🔧 Configurable CNN model
class FlexibleCNN(LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters()

        in_channels = 3  # RGB images
        conv_layers = []
        for i in range(5):
            conv_layers.append(
                nn.Conv2d(
                    in_channels=in_channels,
                    out_channels=config['conv_filters'][i],
                    kernel_size=config['kernel_size'],  # Changed from conv_kernels to kernel_size
                    padding=config.get('conv_padding', 1)
                )
            )
            conv_layers.append(config['activation_fn']())
            conv_layers.append(nn.MaxPool2d(2))
            in_channels = config['conv_filters'][i]

        self.conv_block = nn.Sequential(*conv_layers)

        # Determine flattened feature size after convolutions
        dummy_input = torch.zeros((1, 3, config['input_size'], config['input_size']))
        with torch.no_grad():
            dummy_out = self.conv_block(dummy_input)
        self.flattened_size = dummy_out.view(1, -1).shape[1]

        self.fc1 = nn.Linear(self.flattened_size, config['dense_units'])
        self.fc1_act = config['activation_fn']()
        self.output = nn.Linear(config['dense_units'], config['num_classes'])

    def forward(self, x):
        x = self.conv_block(x)
        x = x.view(x.size(0), -1)
        x = self.fc1_act(self.fc1(x))
        return self.output(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = F.cross_entropy(preds, y)
        acc = (preds.argmax(1) == y).float().mean()
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = F.cross_entropy(preds, y)
        acc = (preds.argmax(1) == y).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

In [ ]:
from pytorch_lightning import Trainer

config = {
    'conv_filters': [32, 64, 128, 128, 256],
    'kernel_size': 3,
    'input_size': 224,
    'activation_fn': nn.ReLU,
    'pool_size': 2,
    'num_classes': 10,
    'dense_units': 512
}

model = FlexibleCNN(config)

trainer = Trainer(
    max_epochs=10,
    accelerator='auto',
    devices=1,
    log_every_n_steps=10
)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
trainer.validate(model, dataloaders=val_loader)  # Optional: print final val accuracy

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type       | Params | Mode 
--------------------------------------------------
0 | conv_block | Sequential | 536 K  | train
1 | fc1        | Linear     | 6.4 M  | train
2 | fc1_act    | ReLU       | 0      | train
3 | output     | Linear     | 5.1 K  | train
--------------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
27.857    Total estimated model params s

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.3714999854564667     │
│         val_loss          │     1.804919719696045     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 1.804919719696045, 'val_acc': 0.3714999854564667}]